<a href="https://colab.research.google.com/github/sk-choi/Yangjae_AI/blob/main/%EC%A0%84%EC%9D%B4%ED%95%99%EC%8A%B5_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.utils import to_categorical
import numpy as np
import pandas as pd


In [ ]:
#불러오는 함수 load_transfer
def load_transfer_model():
  #base_model에 모델 불러오기, input_shape = (96, 96, 3) #cifar10데이터에 적합한 값
  #include_top = False, weights = 'imagenet'
  base_model = ResNet50(input_shape = (84,84,3), include_top = False, weights = 'imagenet')#(96,96,3)에서 (84,84,1)으로 변경
  #base_model이 훈련 가능하지 않게 설정
  base_model.trainable = False
  num_classes = 10
  #layer를 앞뒤로 추가해줍니다.
  transfer_model = Sequential([
        layers.UpSampling2D(size = (3,3), interpolation = 'bilinear'),#size =  ,interpolation = 
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(num_classes, activation = 'softmax')
  ])
  return transfer_model
  #교수님이 권장하는 방법은 input_shape를 조절하는 것이다.

In [ ]:
#include_top을 True로 수정, input_shape를 (224, 224, 3)으로 수정

#rgb채널에는 채널 이름이 있는 것이 아닌 위치만 주어져 있다.
#밝기정보만 주어져있기에 위치에 따라 레드, 블루, 그린이라고 볼 수 있다.
#그래서 위의 정보를 레드정보라고 생각할 수 있다.
#그렇기에 레드, 블루, 그린 각각의 정보를 연결해보는 걸 시도해 볼 수 있다.

In [ ]:
def main(transfer_model = None, epochs = 3):
  #원본 코드는 cifar10 데이터 셋을 내장했었습니다. 그런데 우리는 fashion_mnist에 적용해볼겁니다.
  dataloader = tf.keras.datasets.fashion_mnist #(x_train, y_train), (x_test, y_test)
  (x_train, y_train), (x_test, y_test) = dataloader.load_data()
  x_train, y_train, x_test, y_test = x_train[:5000], y_train[:5000], x_test[:5000], y_test[:5000]
  
  x_train_reshape = x_train.reshape(-1,28,28,1)
  # x_train = np.expand_dims(x_train, -1) #데이터 크기가 (28,28)이면 뒤에 rgb정보가 들어있다
  # x_test = np.expand_dims(x_test, -1)
  x_train_red = x_train_reshape.copy()
  x_train_green = x_train_reshape.copy()
  x_train_blue = x_train_reshape.copy()
  
  x = np.block([x_train_red, x_train_green, x_train_blue])

  one_hot = to_categorical(y_train, 10)
  print('x_train.shape', x_train_reshape.shape)
  print('y_train.shape', y_train.shape)
  print('x', x.shape)
  print('one_hot', one_hot)
  #transfer_model 없을 땐 불러오고, 있으면 넘어가는 코드 만들기
  
  if transfer_model is None:
    transfer_model = load_transfer_model()
  #optimizer, loss, metrics 지정
  optimizer = Adam(learning_rate = 0.001)
  transfer_model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['acc'])

  #모델을 학습
  hist = transfer_model.fit(x, one_hot, epochs = epochs, batch_size = 500)
  #완성된 모델 확인 summary
  transfer_model.summary


  #테스트 데이터로 모델 최종 성능 evaluate
  loss, accuracy = transfer_model.evaluate(x, one_hot)#변수 이름 바꾸기 x, one_hot
  print('accuracy: ', accuracy * 100)

main()

x_train.shape (5000, 28, 28, 1)
y_train.shape (5000,)
x (5000, 28, 28, 3)
one_hot [[0. 0. 0. ... 0. 0. 1.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Epoch 1/3
10/10 [==============================] - 119s 11s/step - loss: 2.1406 - acc: 0.3360
Epoch 2/3
10/10 [==============================] - 112s 11s/step - loss: 0.8670 - acc: 0.7136
Epoch 3/3
157/157 [==============================] - 123s 773ms/step - loss: 0.5726 - acc: 0.7992
79.91999983787537
